## Renting an apartment in New York City.

This notebook is for IBM Applied Data Science Capstone project. In this notebook, we collect the neighborhoods in New York City and analyze the venue distributions in them. The neighborhoods are clustered based on their venues.

#### Download the Dependencies

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [2]:
!pip install folium
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
from bs4 import BeautifulSoup
print('Libraries imported.')

Libraries imported.


#### Extract the data of New York City neighborhoods

In [3]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


#### Take a look at the data

In [4]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

#### Scrape the relevant data

In [5]:
NYC_neighborhoods_data = newyork_data['features']
NYC_neighborhoods_data

[{'type': 'Feature',
  'id': 'nyu_2451_34572.1',
  'geometry': {'type': 'Point',
   'coordinates': [-73.84720052054902, 40.89470517661]},
  'geometry_name': 'geom',
  'properties': {'name': 'Wakefield',
   'stacked': 1,
   'annoline1': 'Wakefield',
   'annoline2': None,
   'annoline3': None,
   'annoangle': 0.0,
   'borough': 'Bronx',
   'bbox': [-73.84720052054902,
    40.89470517661,
    -73.84720052054902,
    40.89470517661]}},
 {'type': 'Feature',
  'id': 'nyu_2451_34572.2',
  'geometry': {'type': 'Point',
   'coordinates': [-73.82993910812398, 40.87429419303012]},
  'geometry_name': 'geom',
  'properties': {'name': 'Co-op City',
   'stacked': 2,
   'annoline1': 'Co-op',
   'annoline2': 'City',
   'annoline3': None,
   'annoangle': 0.0,
   'borough': 'Bronx',
   'bbox': [-73.82993910812398,
    40.87429419303012,
    -73.82993910812398,
    40.87429419303012]}},
 {'type': 'Feature',
  'id': 'nyu_2451_34572.3',
  'geometry': {'type': 'Point',
   'coordinates': [-73.82780644716412, 

#### Transform the data into a pandas dataframe for all of the neighborhoods of New York City

In [6]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
NYC_neighborhoods = pd.DataFrame(columns=column_names)

for data in NYC_neighborhoods_data:
    borough = NYC_neighborhood_name = data['properties']['borough'] 
    NYC_neighborhood_name = data['properties']['name']
        
    NYC_neighborhood_latlon = data['geometry']['coordinates']
    NYC_neighborhood_lat = NYC_neighborhood_latlon[1]
    NYC_neighborhood_lon = NYC_neighborhood_latlon[0]
    
    NYC_neighborhoods = NYC_neighborhoods.append({'Borough': borough,
                                          'Neighborhood': NYC_neighborhood_name,
                                          'Latitude': NYC_neighborhood_lat,
                                          'Longitude': NYC_neighborhood_lon}, ignore_index=True)
NYC_neighborhoods.head(15)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


#### Let's check the size of the dataframe for the New York City neighborhoods

In [7]:
print('The total number of rows in the dataframe for the New York City neighborhoods is', len(NYC_neighborhoods.index))

The total number of rows in the dataframe for the New York City neighborhoods is 306


#### Use geopy library to get the latitude and longitude values of New York City.

In [8]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Create a map of New York City with neighborhoods.

In [9]:
# create map of New York using latitude and longitude values
map_NYC = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(NYC_neighborhoods['Latitude'], NYC_neighborhoods['Longitude'], NYC_neighborhoods['Borough'], NYC_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NYC)  
    
map_NYC

#### Define Foursquare Credentials and Version

In [1]:
# Set up Foursquare API
# To run the following, replace *** with the correct info.
CLIENT_ID = '***' # your Foursquare ID
CLIENT_SECRET = '***' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: ***
CLIENT_SECRET:***


#### Let's explore the neighborhoods in New York City

In [11]:
def getNearbyVenuesbyBorough(boroughs, names, latitudes, longitudes, radius=500, LIMIT=10000):
    
    venues_list=[]
    for borough, name, lat, lng in zip(boroughs, names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            borough,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough',
                  'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Create a new dataframe called *NYC_venues*

In [12]:
NYC_venues = getNearbyVenuesbyBorough(boroughs=NYC_neighborhoods['Borough'],
                                      names=NYC_neighborhoods['Neighborhood'],
                                   latitudes=NYC_neighborhoods['Latitude'],
                                  longitudes=NYC_neighborhoods['Longitude'], 
                                 
                                )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

#### Let's check how many venues were returned for all of New York City

In [13]:
print(NYC_venues.shape)
NYC_venues.head(15)

(10306, 8)


,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bronx,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Bronx,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Bronx,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Bronx,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898276,-73.850381,Caribbean Restaurant
4,Bronx,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station
5,Bronx,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop
6,Bronx,Wakefield,40.894705,-73.847201,SUBWAY,40.890656,-73.849192,Sandwich Place
7,Bronx,Wakefield,40.894705,-73.847201,Baychester Avenue Food Truck,40.892293,-73.843230,Food Truck
8,Bronx,Co-op City,40.874294,-73.829939,Capri II Pizza,40.876374,-73.829940,Pizza Place
9,Bronx,Co-op City,40.874294,-73.829939,Dollar Tree,40.870125,-73.828989,Discount Store


#### Let's check to see which borough has the most venues

In [14]:
number_NYC_venues = NYC_venues.groupby('Borough').count()
number_NYC_venues = number_NYC_venues.drop(['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 
                           'Venue Latitude', 'Venue Longitude'], axis = 1)
number_NYC_venues = number_NYC_venues.rename(columns={'Venue Category': 'Number of Venues'})
number_NYC_venues = number_NYC_venues.sort_values(by='Number of Venues', ascending=False)
number_NYC_venues

,Number of Venues
Borough,
Manhattan,3324
Brooklyn,2812
Queens,2129
Bronx,1243
Staten Island,798


As we can see, Manhattan has the most venues with 3,324.

#### Let's check to see how many restaurants there are in each borough

We do this by finding out how many of the venues in each category contain the word "restaurant" for each borough.

In [15]:
restaurants=NYC_venues[['Restaurant' in s for s in NYC_venues['Venue Category']]]
restaurants
number_restaurants_borough = restaurants.groupby('Borough').count()
number_restaurants_borough = number_restaurants_borough.drop(['Neighborhood', 'Neighborhood Latitude',
                                                      'Neighborhood Longitude', 'Venue',
                                                      'Venue Latitude', 'Venue Longitude'], axis = 1)
number_restaurants_borough = number_restaurants_borough.rename(columns={'Venue Category': 'Number of Restaurants'})
number_restaurants_borough = number_restaurants_borough.sort_values(by='Number of Restaurants', ascending=False)
number_restaurants_borough

,Number of Restaurants
Borough,
Manhattan,932
Brooklyn,661
Queens,582
Bronx,248
Staten Island,161


We see that Manhattan has the most restaurants with 932 for the entire borough. Keep in mind that this might not include pizza parlors, sandwich shops, diners nor certain fast food places.  Since we are only finding the names with the word "restaurant" in it. 

#### Let's check to see how many cocktail bars there are in each borough

We do this by finding out how many of the venues in each category contain the words "Cocktail Bar" for each borough.

In [16]:
cocktail_bars_borough = NYC_venues[['Cocktail Bar' in s for s in NYC_venues['Venue Category']]]
cocktail_bars_borough
number_cocktail_bars_borough = cocktail_bars_borough.groupby('Borough').count()
number_cocktail_bars_borough = number_cocktail_bars_borough.drop(['Neighborhood', 'Neighborhood Latitude',
                                                      'Neighborhood Longitude', 'Venue',
                                                      'Venue Latitude', 'Venue Longitude'], axis = 1)
number_cocktail_bars_borough = number_cocktail_bars_borough.rename(columns={'Venue Category': 'Number of Cocktail Bars'})
number_cocktail_bars_borough = number_cocktail_bars_borough.sort_values(by='Number of Cocktail Bars', ascending=False)
number_cocktail_bars_borough

,Number of Cocktail Bars
Borough,
Manhattan,57
Brooklyn,38
Queens,4
Staten Island,1


We see that Manhattan has the most cocktail bars with 57 in the entire borough.

#### Let's check to see how many museums there are in each borough

In [17]:
museums_borough = NYC_venues[['Museum' in s for s in NYC_venues['Venue Category']]]
museums_borough
number_museums_borough = museums_borough.groupby('Borough').count()
number_museums_borough = number_museums_borough.drop(['Neighborhood', 'Neighborhood Latitude',
                                                      'Neighborhood Longitude', 'Venue',
                                                      'Venue Latitude', 'Venue Longitude'], axis = 1)
number_museums_borough = number_museums_borough.rename(columns={'Venue Category': 'Number of Museums'})
number_museums_borough = number_museums_borough.sort_values(by='Number of Museums', ascending=False)
number_museums_borough

,Number of Museums
Borough,
Manhattan,16
Brooklyn,9
Staten Island,4
Bronx,3
Queens,3


We see that Manhattan has the most museums with 16 in the entire borough.

Manhattan has the most restaurants, cocktail Bars, and museums

#### Let's explore the Neighborhoods in Manhattan. 

In [18]:
manhattan_neighborhoods = NYC_neighborhoods[NYC_neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_neighborhoods.head(15)

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688
5,Manhattan,Manhattanville,40.816934,-73.957385
6,Manhattan,Central Harlem,40.815976,-73.943211
7,Manhattan,East Harlem,40.792249,-73.944182
8,Manhattan,Upper East Side,40.775639,-73.960508
9,Manhattan,Yorkville,40.775930,-73.947118


#### Let's check the size of the dataframe for the Manhattan neighborhoods

Let's get the geographical coordinates of Manhattan.

In [19]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


As we did with all of New York City, let's visualize Manhattan with all of the neighborhoods in it.

In [20]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_neighborhoods['Latitude'], manhattan_neighborhoods['Longitude'], manhattan_neighborhoods['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [21]:
print('The total number of rows in the dataframe for the Manhattan neighborhoods is', len(manhattan_neighborhoods.index))

The total number of rows in the dataframe for the Manhattan neighborhoods is 40


In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Create a new dataframe called *Manhattan_venues*


In [23]:
manhattan_venues = getNearbyVenues(names=manhattan_neighborhoods['Neighborhood'],
                                   latitudes=manhattan_neighborhoods['Latitude'],
                                  longitudes=manhattan_neighborhoods['Longitude'], 
                                 
                                )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [24]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3324, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop


#### Let's check how many venues were returned for each neighborhood in Manhattan


In [25]:
number_manhattan_venues = manhattan_venues.groupby('Neighborhood').count()
number_manhattan_venues = number_manhattan_venues.drop(['Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 
                           'Venue Latitude', 'Venue Longitude'], axis = 1)
number_manhattan_venues = number_manhattan_venues.rename(columns={'Venue Category': 'Number of Venues'})
number_manhattan_venues = number_manhattan_venues.sort_values(by='Number of Venues', ascending=False)
number_manhattan_venues

,Number of Venues
Neighborhood,
Battery Park City,100
Lenox Hill,100
West Village,100
Upper West Side,100
Upper East Side,100
Turtle Bay,100
Tribeca,100
Sutton Place,100
Soho,100


As we can see there are a lot of neighborhoods that have 100 venues which is the most for a neighborhood.

#### Let's check to see how many restaurants there are in each neighborhood in Manhattan

In [26]:
restaurants_neighborhood=manhattan_venues[['Restaurant' in s for s in manhattan_venues['Venue Category']]]
restaurants_neighborhood
number_restaurants_neighborhood = restaurants_neighborhood.groupby('Neighborhood').count()
number_restaurants_neighborhood= number_restaurants_neighborhood.drop(['Neighborhood Latitude',
                                                      'Neighborhood Longitude', 'Venue',
                                                      'Venue Latitude', 'Venue Longitude'], axis = 1)
number_restaurants_neighborhood = number_restaurants_neighborhood.rename(columns={'Venue Category': 'Number of Restaurants'})
number_restaurants_neighborhood = number_restaurants_neighborhood.sort_values(by='Number of Restaurants', ascending=False)
number_restaurants_neighborhood

,Number of Restaurants
Neighborhood,
Chinatown,43
Greenwich Village,42
Turtle Bay,39
East Village,38
Upper West Side,36
Murray Hill,35
West Village,34
Noho,31
Midtown South,31


Chinatown has the most number of restaurants with 43.  Keep in mind that this might not include pizza parlors, sandwich shops, diners nor certain fast food places.  Since we are only finding the names with the word "restaurant" in it. 

#### Let's check to see how many cocktail bars there are in each neighborhood in Manhattan

In [27]:
cocktail_bars_neighborhood=manhattan_venues[['Cocktail Bar' in s for s in manhattan_venues['Venue Category']]]
cocktail_bars_neighborhood
number_cocktail_bars_neighborhood = cocktail_bars_neighborhood.groupby('Neighborhood').count()
number_cocktail_bars_neighborhood = number_cocktail_bars_neighborhood.drop(['Neighborhood Latitude',
                                                      'Neighborhood Longitude', 'Venue',
                                                      'Venue Latitude', 'Venue Longitude'], axis = 1)
number_cocktail_bars_neighborhood = number_cocktail_bars_neighborhood.rename(columns={'Venue Category': 'Number of Cocktail Bars'})
number_cocktail_bars_neighborhood = number_cocktail_bars_neighborhood.sort_values(by='Number of Cocktail Bars', ascending=False)
number_cocktail_bars_neighborhood

,Number of Cocktail Bars
Neighborhood,
Noho,5
Chinatown,4
East Village,4
Gramercy,4
Midtown,4
Civic Center,3
Upper East Side,3
Midtown South,3
Lenox Hill,2


We can see that NoHo has the most Cocktail Bars with 5

#### Let's check to see how many Museums there are in each neighborhood in Manhattan

In [28]:
museums_neighborhood=manhattan_venues[['Museum' in s for s in manhattan_venues['Venue Category']]]
museums_neighborhood
number_museums_neighborhood = museums_neighborhood.groupby('Neighborhood').count()
number_museums_neighborhood = number_museums_neighborhood.drop(['Neighborhood Latitude',
                                                      'Neighborhood Longitude', 'Venue',
                                                      'Venue Latitude', 'Venue Longitude'], axis = 1)
number_museums_neighborhood = number_museums_neighborhood.rename(columns={'Venue Category': 'Number of Museums'})
number_museums_neighborhood = number_museums_neighborhood.sort_values(by='Number of Museums', ascending=False)
number_museums_neighborhood

,Number of Museums
Neighborhood,
Carnegie Hill,2
Soho,2
Upper East Side,2
Chinatown,1
Civic Center,1
Financial District,1
Inwood,1
Little Italy,1
Manhattanville,1


We can see that Carnegie Hill, Soho, and the Upper East Side have 2 museums which is the most per neighborhood.

#### Let's analyze the venues in each neighborhood

In [29]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

#move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head(15)

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Gym,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Ski Shop,Sm

### Let's examine the new dataframe size.

In [30]:
manhattan_onehot.shape

(3324, 333)

#### Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [31]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Gym,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Ski Shop,Sm

#### Let's confirm the new size

In [32]:
manhattan_grouped.shape

(40, 333)

#### Let's print each neighborhood along with the top 7 most common venues

In [33]:
num_top_venues = 7

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
                venue  freq
0                Park  0.08
1         Coffee Shop  0.07
2               Hotel  0.05
3       Memorial Site  0.04
4  Italian Restaurant  0.03
5      Clothing Store  0.03
6                 Gym  0.03


----Carnegie Hill----
         venue  freq
0  Pizza Place  0.06
1  Coffee Shop  0.06
2         Café  0.04
3  Yoga Studio  0.03
4          Gym  0.03
5          Bar  0.03
6    Bookstore  0.03


----Central Harlem----
                  venue  freq
0    African Restaurant  0.06
1    Chinese Restaurant  0.06
2   Fried Chicken Joint  0.04
3            Public Art  0.04
4    Seafood Restaurant  0.04
5  Gym / Fitness Center  0.04
6        Cosmetics Shop  0.04


----Chelsea----
                 venue  freq
0          Coffee Shop  0.06
1   Italian Restaurant  0.05
2       Ice Cream Shop  0.05
3               Bakery  0.04
4  American Restaurant  0.04
5            Nightclub  0.04
6                Hotel  0.03


----Chinatown----
                   venu

### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Coffee Shop,Hotel,Memorial Site,Wine Shop,Italian Restaurant,Gym,Clothing Store,Plaza,Burger Joint
1,Carnegie Hill,Coffee Shop,Pizza Place,Café,Yoga Studio,Bookstore,Wine Shop,Cosmetics Shop,French Restaurant,Bar,Japanese Restaurant
2,Central Harlem,African Restaurant,Chinese Restaurant,Public Art,Fried Chicken Joint,Seafood Restaurant,American Restaurant,Pizza Place,Cosmetics Shop,Gym / Fitness Center,French Restaurant
3,Chelsea,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bakery,Nightclub,American Restaurant,Seafood Restaurant,Theater,Hotel,Gift Shop
4,Chinatown,Chinese Restaurant,American Restaurant,Cocktail Bar,Dumpling Restaurant,Bubble Tea Shop,Spa,Vietnamese Restaurant,Asian Restaurant,Jewelry Store,Bakery
5,Civic Center,Italian Restaurant,Gym / Fitness Center,Sandwich Place,French Restaurant,Bakery,Coffee Shop,Hotel,Sporting Goods Shop,Spa,Park
6,Clinton,Theater,Gym / Fitness Center,Italian Restaurant,American Restaurant,Hotel,Wine Shop,Spa,Coffee Shop,Dog Run,Lounge
7,East Harlem,Mexican Restaurant,Bakery,Deli / Bodega,Spa,Thai Restaurant,Latin American Restaurant,Convenience Store,Bank,Coffee Shop,Cocktail Bar
8,East Village,Bar,Wine Bar,Mexican Restaurant,Ice Cream Shop,Chinese Restaurant,Cocktail Bar,Coffee Shop,Vegetarian / Vegan Restaurant,Ramen Restaurant,Pizza Place
9,Financial District,Coffee Shop,Steakhouse,Wine Shop,Hotel,Gym,Gym / Fitness Center,American Restaurant,Café,Bar,Pizza Place


#### Let's Cluster the neighborhoods

Run *k*-means to cluster the neighborhood into 6 clusters.

In [36]:
# set number of clusters
kclusters = 6

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 4, 0, 3, 0, 0, 1, 3, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [37]:

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,4,Discount Store,Sandwich Place,Coffee Shop,Yoga Studio,Pizza Place,Tennis Stadium,Supplement Shop,Steakhouse,Seafood Restaurant,Clothing Store
1,Manhattan,Chinatown,40.715618,-73.994279,3,Chinese Restaurant,American Restaurant,Cocktail Bar,Dumpling Restaurant,Bubble Tea Shop,Spa,Vietnamese Restaurant,Asian Restaurant,Jewelry Store,Bakery
2,Manhattan,Washington Heights,40.851903,-73.936900,4,Café,Mobile Phone Shop,Bakery,Mexican Restaurant,Pizza Place,Sandwich Place,Grocery Store,New American Restaurant,Supplement Shop,Gym
3,Manhattan,Inwood,40.867684,-73.921210,4,Mexican Restaurant,Café,Lounge,Bakery,Pizza Place,Deli / Bodega,Park,Chinese Restaurant,Frozen Yogurt Shop,Pharmacy
4,Manhattan,Hamilton Heights,40.823604,-73.949688,4,Pizza Place,Café,Mexican Restaurant,Coffee Shop,Yoga Studio,Bakery,Cocktail Bar,Sandwich Place,School,Chinese Restaurant
5,Manhattan,Manhattanville,40.816934,-73.957385,4,Chinese Restaurant,Coffee Shop,Park,Mexican Restaurant,Seafood Restaurant,Italian Restaurant,Burger Joint,Supermarket,Other Nightlife,Grocery Store
6,Manhattan,Central Harlem,40.815976,-73.943211,4,African Restaurant,Chinese Restaurant,Public Art,Fried Chicken Joint,Seafood Restaurant,American Restaurant,Pizza Place,Cosmetics Shop,Gym / Fitness Center,French Restaurant
7,Manhattan,East Harlem,40.792249,-73.944182,1,Mexican Restaurant,Bakery,Deli / Bodega,Spa,Thai Restaurant,Latin American Restaurant,Convenience Store,Bank,Coffee Shop,Cocktail Bar
8,Manhattan,Upper East Side,40.775639,-73.960508,0,Exhibit,Italian Restaurant,Coffee Shop,Gym / Fitness Center,Juice Bar,Bakery,Art Gallery,French Restaurant,Cocktail Bar,Spa
9,Manhattan,Yorkville,40.775930,-73.947118,0,Italian Restaurant,Bar,Coffee Shop,Gym,Sushi Restaurant,Pizza Place,Mexican Restaurant,Ice Cream Shop,Deli / Bodega,Diner


Let's visualize the resulting clusters

In [38]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Let's examine the clusters

##### Cluster 0

In [39]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Upper East Side,Exhibit,Italian Restaurant,Coffee Shop,Gym / Fitness Center,Juice Bar,Bakery,Art Gallery,French Restaurant,Cocktail Bar,Spa
9,Yorkville,Italian Restaurant,Bar,Coffee Shop,Gym,Sushi Restaurant,Pizza Place,Mexican Restaurant,Ice Cream Shop,Deli / Bodega,Diner
10,Lenox Hill,Sushi Restaurant,Italian Restaurant,Coffee Shop,Pizza Place,Sporting Goods Shop,Gym / Fitness Center,Burger Joint,Gym,Café,Mexican Restaurant
12,Upper West Side,Italian Restaurant,Wine Bar,Bar,Coffee Shop,Mediterranean Restaurant,Bakery,Indian Restaurant,Vegetarian / Vegan Restaurant,Yoga Studio,Pub
13,Lincoln Square,Gym / Fitness Center,Theater,Café,Concert Hall,Plaza,Italian Restaurant,Opera House,Performing Arts Venue,French Restaurant,Indie Movie Theater
14,Clinton,Theater,Gym / Fitness Center,Italian Restaurant,American Restaurant,Hotel,Wine Shop,Spa,Coffee Shop,Dog Run,Lounge
15,Midtown,Hotel,Coffee Shop,Cocktail Bar,Clothing Store,Theater,Bookstore,Steakhouse,Food Truck,Spa,Bakery
16,Murray Hill,Hotel,Coffee Shop,Sandwich Place,Japanese Restaurant,Gym,Salon / Barbershop,Italian Restaurant,French Restaurant,Cocktail Bar,Bagel Shop
17,Chelsea,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bakery,Nightclub,American Restaurant,Seafood Restaurant,Theater,Hotel,Gift Shop
18,Greenwich Village,Italian Restaurant,Clothing Store,Sushi Restaurant,Indian Restaurant,Café,French Restaurant,Ice Cream Shop,Seafood Restaurant,Gourmet Shop,Spa


##### Cluster 1

In [40]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,East Harlem,Mexican Restaurant,Bakery,Deli / Bodega,Spa,Thai Restaurant,Latin American Restaurant,Convenience Store,Bank,Coffee Shop,Cocktail Bar
26,Morningside Heights,Park,American Restaurant,Coffee Shop,Bookstore,Burger Joint,Food Truck,Deli / Bodega,Tennis Court,Seafood Restaurant,Salad Place
36,Tudor City,Park,Mexican Restaurant,Café,Greek Restaurant,Asian Restaurant,Deli / Bodega,Pizza Place,Hotel,Dog Run,Spa


##### Cluster 2

In [41]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Stuyvesant Town,Park,Bar,Playground,Boat or Ferry,Pet Service,Baseball Field,Fountain,Harbor / Marina,German Restaurant,Cocktail Bar


##### Cluster 3

In [42]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,American Restaurant,Cocktail Bar,Dumpling Restaurant,Bubble Tea Shop,Spa,Vietnamese Restaurant,Asian Restaurant,Jewelry Store,Bakery
19,East Village,Bar,Wine Bar,Mexican Restaurant,Ice Cream Shop,Chinese Restaurant,Cocktail Bar,Coffee Shop,Vegetarian / Vegan Restaurant,Ramen Restaurant,Pizza Place


##### Cluster 4

In [43]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Discount Store,Sandwich Place,Coffee Shop,Yoga Studio,Pizza Place,Tennis Stadium,Supplement Shop,Steakhouse,Seafood Restaurant,Clothing Store
2,Washington Heights,Café,Mobile Phone Shop,Bakery,Mexican Restaurant,Pizza Place,Sandwich Place,Grocery Store,New American Restaurant,Supplement Shop,Gym
3,Inwood,Mexican Restaurant,Café,Lounge,Bakery,Pizza Place,Deli / Bodega,Park,Chinese Restaurant,Frozen Yogurt Shop,Pharmacy
4,Hamilton Heights,Pizza Place,Café,Mexican Restaurant,Coffee Shop,Yoga Studio,Bakery,Cocktail Bar,Sandwich Place,School,Chinese Restaurant
5,Manhattanville,Chinese Restaurant,Coffee Shop,Park,Mexican Restaurant,Seafood Restaurant,Italian Restaurant,Burger Joint,Supermarket,Other Nightlife,Grocery Store
6,Central Harlem,African Restaurant,Chinese Restaurant,Public Art,Fried Chicken Joint,Seafood Restaurant,American Restaurant,Pizza Place,Cosmetics Shop,Gym / Fitness Center,French Restaurant
20,Lower East Side,Coffee Shop,Ramen Restaurant,Café,Pizza Place,Cocktail Bar,Sandwich Place,Chinese Restaurant,Art Gallery,Bakery,Park
25,Manhattan Valley,Indian Restaurant,Coffee Shop,Pizza Place,Yoga Studio,Mexican Restaurant,Café,Bar,Thai Restaurant,Deli / Bodega,Szechuan Restaurant
30,Carnegie Hill,Coffee Shop,Pizza Place,Café,Yoga Studio,Bookstore,Wine Shop,Cosmetics Shop,French Restaurant,Bar,Japanese Restaurant


##### Cluster 5

In [44]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 5, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Roosevelt Island,Park,Coffee Shop,Sandwich Place,Playground,Bus Stop,Farmers Market,Supermarket,Outdoors & Recreation,Dry Cleaner,School
